In [ ]:
# default_exp models.ResCNN

# ResCNN

> This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com based on:
* Zou, X., Wang, Z., Li, Q., & Sheng, W. (2019). Integration of residual network and convolutional neural network along with various activation functions and global pooling for time series classification. Neurocomputing.
* No official implementation found

In [ ]:
#export
from tsai.imports import *
from tsai.models.layers import *

In [ ]:
#export
class Block(Module):
    def __init__(self, ni, nf, ks=[7, 5, 3], act_fn='relu'):
        self.conv1 = Conv1d(ni, nf, ks[0], padding='same', act_fn=act_fn)
        self.conv2 = Conv1d(nf, nf, ks[1], padding='same', act_fn=act_fn)
        self.conv3 = Conv1d(nf, nf, ks[2], padding='same', act_fn=False)

        # expand channels for the sum if necessary
        self.shortcut = noop if ni == nf else Conv1d(ni, nf, ks=1, act_fn=False)
        self.act_fn = get_act_layer(act_fn)

    def forward(self, x):
        res = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        sc = self.shortcut(res)
        x += sc
        x = self.act_fn(x)
        return x


class ResCNN(Module):
    def __init__(self, c_in, c_out):
        nf = 64
        self.block = Block(c_in, nf, ks=[7, 5, 3], act_fn='relu')
        self.conv1 = Conv1d(nf, nf * 2, ks=3, padding='same', act_fn='leakyrelu', act_kwargs={'negative_slope':.2})
        self.conv2 = Conv1d(nf * 2, nf * 4, ks=3, padding='same', act_fn='prelu')
        self.conv3 = Conv1d(nf * 4, nf * 2, ks=3, padding='same', act_fn='elu', act_kwargs={'alpha':.3})
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.lin = nn.Linear(nf * 2, c_out)

    def forward(self, x):
        x = self.block(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.gap(x).squeeze(-1)
        return self.lin(x)

In [ ]:
xb = torch.rand(16, 3, 128)
test_eq(ResCNN(3,2)(xb).shape, [xb.shape[0], 2])
ResCNN(3,2)

ResCNN(
  (block): Block(
    (conv1): Sequential(
      (0): Conv1dSame(
        (conv1d_same): Conv1d(3, 64, kernel_size=(7,), stride=(1,))
      )
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv1dSame(
        (conv1d_same): Conv1d(64, 64, kernel_size=(5,), stride=(1,))
      )
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv3): Sequential(
      (0): Conv1dSame(
        (conv1d_same): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
      )
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (shortcut): Sequential(
      (0): Conv1dSame(
        (conv1d_same): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      )
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act_fn): ReLU()
  )
  (conv1): Sequential(
 

In [ ]:
#hide
out = create_scripts()
beep(out)